In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan.model import DATGAN
#from modules.datgan_preLSTM.model import DATGAN

#from modules.datgan_no_att.model import DATGAN
import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/DATGAN_asd29/'

In [10]:
datgan = DATGAN(continuous_columns, max_epoch=1500, batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1019 08:46:43 @model.py:159] Found preprocessed data
[1019 08:46:43 @model.py:167] Preprocessed data have been loaded!



[1019 08:46:43 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[1019 08:46:43 @graph.py:277] Creating cell for age (in-edges: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1019 08:46:43 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1019 08:46:43 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[1019 08:46:43 @registry.py:126] gen/LSTM/00/FC2_val input: [500, 100]
[1019 08:46:43 @registry.py:134] gen/LSTM/00/FC2_val output: [500, 6]
[1019 08:46:43 @registry.py:12

[1019 08:46:44 @registry.py:134] gen/LSTM/06/FC3 output: [500, 100]
[1019 08:46:44 @graph.py:277] Creating cell for hh_income (in-edges: 2)
[1019 08:46:44 @registry.py:126] gen/LSTM/07/FC_inputs input: [500, 200]
[1019 08:46:44 @registry.py:134] gen/LSTM/07/FC_inputs output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/07/FC_attentions input: [500, 200]
[1019 08:46:44 @registry.py:134] gen/LSTM/07/FC_attentions output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/07/FC_lstm_state_0 input: [500, 200]
[1019 08:46:44 @registry.py:134] gen/LSTM/07/FC_lstm_state_0 output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/07/FC_lstm_state_1 input: [500, 200]
[1019 08:46:44 @registry.py:134] gen/LSTM/07/FC_lstm_state_1 output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/08/FC input: [500, 100]
[1019 08:46:44 @registry.py:134] gen/LSTM/08/FC output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/08/FC2 input: [500, 100]
[1019 08:46:44 @registry.py:134] gen/LST

[1019 08:46:44 @graph.py:277] Creating cell for choice (in-edges: 7)
[1019 08:46:44 @registry.py:126] gen/LSTM/20/FC_inputs input: [500, 700]
[1019 08:46:44 @registry.py:134] gen/LSTM/20/FC_inputs output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/20/FC_attentions input: [500, 700]
[1019 08:46:44 @registry.py:134] gen/LSTM/20/FC_attentions output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/20/FC_lstm_state_0 input: [500, 700]
[1019 08:46:44 @registry.py:134] gen/LSTM/20/FC_lstm_state_0 output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/20/FC_lstm_state_1 input: [500, 700]
[1019 08:46:44 @registry.py:134] gen/LSTM/20/FC_lstm_state_1 output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/21/FC input: [500, 100]
[1019 08:46:44 @registry.py:134] gen/LSTM/21/FC output: [500, 100]
[1019 08:46:44 @registry.py:126] gen/LSTM/21/FC2 input: [500, 100]
[1019 08:46:44 @registry.py:134] gen/LSTM/21/FC2 output: [500, 5]
[1019 08:46:44 @registry.py:126] gen/LSTM/21/

[1019 08:46:49 @base.py:209] Setup callbacks graph ...






[1019 08:46:49 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1019 08:46:49 @summary.py:93] Summarizing collection 'summaries' of size 9.

[1019 08:46:49 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.



[1019 08:46:50 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[1019 08:46:52 @base.py:236] Initializing the session ...
[1019 08:46:52 @sessinit.py:114] Restoring checkpoint from ../output/Chicago/DATGAN_asd29/model\model-175000 ...
INFO:tensorflow:Restoring parameters from ../output/Chicago/DATGAN_asd29/model\model-175000
[1019 08:46:53 @base.py:243] Graph Finalized.


[1019 08:46:53 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...
[1019 08:46:53 @steps.py:124] Start training with global_step=175000
[1019 08:46:53 @monitor.py:347] Fou

 92%|############################################################################3      |161/175[00:07<00:00,29.67it/s]

...

[1019 09:35:08 @base.py:285] Epoch 1499 (global_step 262325) finished, time:5.49 seconds.
[1019 09:35:08 @saver.py:79] Model saved to ../output/Chicago/DATGAN_asd29/model\model-262325.


[1019 09:35:08 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.778
[1019 09:35:08 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.46
[1019 09:35:08 @monitor.py:467] GAN_loss/discrim/loss: 0.59842
[1019 09:35:08 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.3812
[1019 09:35:08 @monitor.py:467] GAN_loss/gen/klloss: 0.054696
[1019 09:35:08 @monitor.py:467] GAN_loss/gen/loss: 1.3265
[1019 09:35:08 @monitor.py:467] QueueInput/queue_size: 50
[1019 09:35:08 @base.py:275] Start Epoch 1500 ...


100%|###################################################################################|175/175[00:05<00:00,31.92it/s]

[1019 09:35:13 @base.py:285] Epoch 1500 (global_step 262500) finished, time:5.48 seconds.
[1019 09:35:13 @saver.py:79] Model saved to ../output/Chicago/DATGAN_asd29/model\model-262500.


[1019 09:35:14 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.838
[1019 09:35:14 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.398
[1019 09:35:14 @monitor.py:467] GAN_loss/discrim/loss: 0.61812
[1019 09:35:14 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.4111
[1019 09:35:14 @monitor.py:467] GAN_loss/gen/klloss: 0.070776
[1019 09:35:14 @monitor.py:467] GAN_loss/gen/loss: 1.3403
[1019 09:35:14 @monitor.py:467] QueueInput/queue_size: 50
[1019 09:35:14 @base.py:289] Training has finished!

[1019 09:35:14 @graph.py:277] Creating cell for age (in-edges: 0)
[1019 09:35:14 @graph.py:277] Creating cell for gender (in-edges: 0)
[1019 09:35:14 @graph.py:277] Creating cell for travel_dow (in-edges: 0)
[1019 09:35:14 @graph.py:277] Creating cell for license (in-edges: 1)
[1019 09:35:14 @graph.py:277] Creating cell for education_level (in-edges: 1)
[1019 09:35:14 @graph.py:277] Creating cell for work_status (in-edges: 2)
[1019 09:35:14 @graph.py:277] Creating cell for hh_income (in-edges: 2

In [12]:
datgan.save('trained', force=True)

[1019 09:35:50 @model.py:352] Model saved successfully.


In [13]:
beepy.beep(6)